# Start

In [ ]:
# Initialization based on the RIXS function used in the IPÊ beamline.
%matplotlib widget
from default import *
calib = ['value','value']
s = rixs('value', sbins='value', calib=calib).set_calib(-1).fix_monotonicity().set_shift('value')

# Define fit_peaks function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import brixs.addons.fitting
import brixs.model

def fit_peaks(spectrum, start=None, stop=None, initial_guesses=None, constraints=None, save=False, filename='fit_result.txt'):
    """
    ---------------------------------------------------------------------------------
    |                       Code written by Giordano Bispo                           |
    ---------------------------------------------------------------------------------
    This function automates the initialization of the BRIXS model, peak generation, 
    constraint application, and visualization. It handles the specific BRIXS internal 
    indexing (i1, i2) automatically, allowing the user to provide simple lists of parameters.
    Args:
        spectrum (brixs.Spectrum): The input spectrum object containing .x (energy) 
            and .y (intensity) data. Must be pre-processed (e.g., zero-shifted).
        start (float, optional): Lower energy limit for the fitting range (ROI). 
            Defaults to min(spectrum.x).
        stop (float, optional): Upper energy limit for the fitting range (ROI). 
            Defaults to max(spectrum.x).
        initial_guesses (list of dict, required): A list of dictionaries, where each 
            dictionary defines the initial parameters for one peak.
            Keys per dictionary:
                - 'amp' (float): Amplitude/Height.
                - 'c' (float): Center position.
                - 'w' (float): Full Width at Half Maximum (FWHM).
                - 'm' (float, optional): Pseudo-Voigt mixing factor. 
                  0.0 = Gaussian, 1.0 = Lorentzian. Defaults to 0.
                - 'm_vary' (bool, optional): Whether to optimize 'm'. Defaults to False.
        constraints (list of dict, optional): A list of dictionaries defining boundary 
            conditions. The list order must match `initial_guesses`.
            Keys per dictionary:
                - 'min' (float): Lower bound for the parameter.
                - 'max' (float): Upper bound for the parameter.
                - 'vary' (bool): Fix (False) or fit (True) the parameter.
                - 'value' (float): Force a specific value (usually used with vary=False).
            Note: Constraints keys must match parameter names (e.g., {'c': {'min': 0}}).
        save (bool, optional): If True, saves the fit statistics and curve data to a 
            text file. Defaults to False.
        filename (str, optional): Path/name of the output file if save=True. 
            Defaults to 'fit_result.txt'.
    Returns:
        str: A formatted string containing the final fit parameters and statistics 
        (output of spectrum.model.pretty_print()).

    Example:
        >>> guesses = [
        ...     {'amp': 10, 'c': 0.0, 'w': 0.2},       # Peak 0 (Elastic)
        ...     {'amp': 5,  'c': 1.5, 'w': 0.5, 'm': 0.5} # Peak 1 (Inelastic)
        ... ]
        >>> rules = [
        ...     {'c': {'vary': False}},                # Fix Peak 0 position
        ...     {'w': {'min': 0.1, 'max': 1.0}}        # Constrain Peak 1 width
        ... ]
        >>> fit_peaks(s, start=-0.5, stop=5, initial_guesses=guesses, constraints=rules)

    Notes:
        - The function internally maps the user-provided list index to BRIXS parameters 
          using the format `{param}1_{index}_0` (e.g., `amp1_0_0` for the first peak).
        - The fit uses the Levenberg-Marquardt algorithm via lmfit.       
    """

    # --------------------------------------------------
    # 1. Data Preparation
    # --------------------------------------------------
    spectrum.model = brixs.model.Model()
    spectrum.model.parent = spectrum
    spectrum.model.peaks.clear()      

    if start is None: start = min(spectrum.x)
    if stop is None: stop = max(spectrum.x)

    # --------------------------------------------------
    # 2. Model Building
    # --------------------------------------------------
    for i, p in enumerate(initial_guesses):
        m_status = p.get('m_vary', False)
        # i1=i (peak index), i2=0 (spectrum index)
        spectrum.model.peaks.add(amp=p['amp'], c=p['c'], w=p['w'], m=p.get('m', 0), 
                                 m_vary=m_status, i1=i, i2=0)

    # --------------------------------------------------
    # 3. Applying Constraints
    # --------------------------------------------------
    if constraints:
        for j, p in enumerate(constraints): 
            for param, settings in p.items():
                # Brixs structure uses param_i1_i2 (e.g., amp1_0_0)
                model_key = f"{param}1_{j}_0"
                if model_key in spectrum.model:
                    param_obj = spectrum.model[model_key]
                    if 'min' in settings: param_obj.min = settings['min']
                    if 'max' in settings: param_obj.max = settings['max']
                    if 'vary' in settings: param_obj.vary = settings['vary']
                    if 'value' in settings: param_obj.value = settings['value']
                else:
                    print(f"Warning: Parameter {model_key} not found.")
    
    # --------------------------------------------------
    # 4. Running the Fit
    # --------------------------------------------------
    # The fit() method in brixs automatically updates spectrum.model with the best values
    output = spectrum.model.fit(limits=(start, stop))
    
    # Handling the return tuple (report, result)
    if isinstance(output, tuple):
        fit_report = output[0]
        result = output[1]     
    else:
        result = output

    # --------------------------------------------------
    # 5. Generating Data (using native BRIXS methods)
    # --------------------------------------------------
    mask = (spectrum.x >= start) & (spectrum.x <= stop)
    x_data = spectrum.x[mask]
    y_data = spectrum.y[mask]
    
    # A) Calculate Total Fit
    # Brixs creates a lambda function with current values. Since fit() updated the
    # model, we just ask for the function for spectrum 0 (i2=0).
    model_function = spectrum.model.get_model(i2=0)
    y_fit = model_function(x_data)
    
    # B) Calculate Individual Components
    comps = {}
    
    # Check if peaks exist and calculate individual curves
    # The method calculate_spectrum_for_each_i1 returns a list of spectrum objects
    if hasattr(spectrum.model, 'peaks') and len(spectrum.model.peaks._get_all_tags()) > 0:
        peaks_spectra = spectrum.model.peaks.calculate_spectrum_for_each_i1(i2=0, x=x_data)
        for k, peak_spec in enumerate(peaks_spectra):
             comps[f'peak_{k}'] = peak_spec.y

    # Residual
    residual = y_data - y_fit

    # --------------------------------------------------
    # 6. Plotting
    # --------------------------------------------------
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [10, 1]}, figsize=(10, 8))
    
    

    ax1.scatter(x_data, y_data, s=15, color='black', alpha=0.6, label='Experimental')
    ax1.plot(x_data, y_fit, 'r-', linewidth=2, label='Total Fit')
    
    for name, y_comp in comps.items():
        ax1.plot(x_data, y_comp, '--', alpha=0.8, label=name)
    
    ax1.set_ylabel('Intensity')
    ax1.legend(loc='best', fontsize='small')
    ax1.grid(alpha=0.3)
    ax1.set_title('Fit Result (BRIXS)')

    ax2.plot(x_data, residual, 'b-', alpha=0.7)
    ax2.axhline(0, color='black', linestyle='--', linewidth=1)
    ax2.set_ylabel('Residual')
    ax2.set_xlabel('Energy Loss (eV)')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

    # --------------------------------------------------
    # 7. Saving to File
    # --------------------------------------------------
    if save:
        header_lines = []
        header_lines.append("=== Fit Result ===")
        header_lines.append(f"Chi-square: {result.chisqr:.4e}")
        header_lines.append(f"Red. Chi-square: {result.redchi:.4e}")
        header_lines.append("-" * 30)
        
        # Iterate over spectrum.model, which is already updated with final values
        for name in spectrum.model:
            param = spectrum.model[name]
            err = param.stderr if param.stderr else 0
            header_lines.append(f"{name:<15}: {param.value:.4f} +/- {err:.4f}")
        
        header_lines.append("-" * 30)
        
        data_matrix = [x_data, y_data, y_fit, residual]
        col_names = "x \t y_obs \t y_calc \t residual"
        
        for name, y_comp in comps.items():
            data_matrix.append(y_comp)
            col_names += f" \t {name}"
        
        save_data = np.column_stack(data_matrix)
        full_header = "\n".join(header_lines) + "\n" + col_names
        
        try:
            np.savetxt(filename, save_data, header=full_header, fmt='%.6e', delimiter='\t')
            print(f"File saved: {filename}")
        except Exception as e:
            print(f"Error saving file: {e}")

    return spectrum.model.pretty_print()

# Example

In [ ]:
"""
Usage Example:
    This block demonstrates the correct implementation of the `fit_peaks` function, 
    focusing on the construction of the initialization (`guesses`) and boundary 
    condition (`constraints`) structures.

    1. Initial Guesses (`guesses`)
    ------------------------------
    A list of dictionaries defining the starting parameters for the optimization algorithm.
    Each dictionary represents a single peak with the following keys:
        - 'amp' (float): Initial amplitude (intensity) of the peak.
        - 'c'   (float): Initial center position (energy/shift).
        - 'w'   (float): Full Width at Half Maximum (FWHM).
        - 'm'   (float): Pseudo-Voigt mixing parameter (0.0 = Pure Gaussian, 1.0 = Pure Lorentzian).
        - 'm_vary' (bool, optional): If True, allows the lineshape mixing parameter to vary 
          during the fit. Defaults to False.

    Note: The `initial_guesses` argument is mandatory. Passing `None` will raise a ValueError.

    2. Constraints (`constraints`)
    ------------------------------
    A list of dictionaries matching the order of `guesses`, defining the boundary conditions 
    for the fitting parameters.
        - 'min'  (float): Sets a lower bound for the parameter.
        - 'max'  (float): Sets an upper bound for the parameter.
        - 'vary' (bool):  If False, fixes the parameter to its initial value. Defaults to True.

    Note: Setting 'min' or 'max' limits implicitly sets 'vary' to True; explicit declaration 
    will raise a ValueError. If `constraints` is None, the fit proceeds with unbounded parameters.
"""
guesses = [
    # Peak 0: Elastic
    {'amp': 1.2, 'c': 0.00, 'w': 0.2, 'm': 0.0, 'm_vary': True},
    # Peak 1: Inelastic
    {'amp': 0.9, 'c': 0.30, 'w': 0.2, 'm': 0.0},
]

constraints = [
    # Constraints for Peak 0
    {
        'amp': {'min': 1.2, 'max': 1.9}, 
        'c':   {'vary': False}, 
        'w':   {'min': 0.2}
    },
    # Constraints for Peak 1
    {
        'amp': {'min': 0.5},
        'c':   {'min': 0.3, 'max': 1.0}
    }
]

# Running the fit
# Execute fit: Requires pre-treated spectrum (zero-shifted/normalized). 
# Filename shall be provided with extension.
fit_peaks(spectrum, start=-1, stop=8, initial_guesses=guesses, constraints=constraints, save=True, filename="rixs_fit_results.txt")